#### Introdução

O principal objetivo deste trabalho era reduzir o tempo necessário para encontrar a frequência de fase de sistemas binários em comparação com o método PDM. O método PDM é altamente eficiente na determinação da frequência de fase e é mais flexível em relação aos dados de entrada, em comparação com outros métodos. No entanto, quando não se tem uma ideia clara de por onde começar a buscar a frequência de fase, esse método pode se tornar computacionalmente custoso.

Inicialmente, a ideia era treinar um algoritmo de regressão para aprender a identificar a relação entre a série temporal do brilho do sistema binário e sua frequência de fase. Entretanto, encontrei alguns desafios ao longo do caminho.
Diversidade dos dados:

Uma das razões para o uso do método PDM é a sua flexibilidade em relação à série de entrada. A Transformada de Fourier, por exemplo, não permite uma série com variação no intervalo de tempo entre as medidas.

No contexto do algoritmo de regressão, a diversidade dos dados também se tornou um desafio, mas consegui superá-lo utilizando processamento dos dados.

Uma alternativa para evitar o processamento extensivo dos dados seria o uso de redes neurais. No entanto, esses modelos são consideravelmente mais complexos de implementar.

#### Poder computacional:

É irônico que o principal desafio tenha sido exatamente o que eu desejava resolver. A questão é que a relação entre a frequência de fase e a série temporal do brilho não parece ser fácil de correlacionar. Isso, somado ao meu modesto notebook, que limitou o tamanho do dataset que eu pude gerar, tornou a tarefa de treinar o algoritmo de regressão bastante complicada.

No entanto, é importante destacar que, embora o treinamento do algoritmo de regressão seja demorado, seu valor reside no fato de que, uma vez treinado, ele poderia reduzir drasticamente o tempo necessário para encontrar a frequência de fase de séries temporais, quase a zero.

#### Resultados obtidos

Apesar de não ter sido possível obter resultados satisfatórios com o uso de algoritmos de regressão, o objetivo inicial do trabalho foi alcançado. Parte do processamento de dados necessário para preparar os dados para o treinamento das IAs envolveu a interpolationolação dos dados. No entanto, ao interpolationolar os dados, conseguimos remover a variação no intervalo de tempo entre as medidas, tornando possível o uso da série de Fourier para calcular a frequência de fase.

Ao utilizar esse método para calcular a frequência, obtive resultados bastante satisfatórios. A média do erro ficou próxima de 1%, variando de acordo com o dataset gerado, mesmo considerando a presença de ruído no sinal. Além disso, em termos de tempo de execução, foi possível reduzir significativamente o tempo necessário em comparação com o método PDM.


### Gerando Dataset

In [ ]:
import numpy as np
import matplotlib.pylab as pl
import pandas as pd
import time

In [ ]:
def sinusoidal(amplitude, w0, size, resolution):
    """ generate a sinusoidal function

    Args:
        amplitude (float): signal amplitude
        w0 (_type_): frequency
        size (_type_): series size
        resolution (_type_): measurement resolution

    Returns:
        list: points of the sinusoidal function
    """
    
    x = np.round((np.sort(np.random.uniform(0, 20*np.pi, size))), resolution)
    
    y = amplitude*np.sin(w0*x) + np.random.normal(0, 0.1, len(x))
    
    return x, y

In [ ]:
def interpolation(x, y, resolution):
    """ interpolates the y points to fill in the gaps

    Args:
        x (list): x axis points of the sinusoidal function
        y (list): y axis points of the sinusoidal function

    Returns:
        list: new interpolated y
    """

    min_value = np.min(x)
    max_value= np.max(x)

    # Creates a new list at regular intervals
    x_novo = np.arange(min_value, max_value, 1/10**resolution)

    # Use interpolation to find the news values of y
    interp_y = np.round(np.interp(x_novo, x, y),resolution)
    
    return interp_y

In [ ]:
'''
Aqui você irá poder alterar os parâmetros para gerar o dataset. Caso queira ver a diferença entre o tempo de execução para calcular a frequência de fase
usando a série de Fourier com os dados interpolados e o método PDM, sugiro deixar os parâmetros como estão. 

Para treinar os algoritmos de machine learning, sugiro reduzir a resolução e aumentar o DataSize, que irá aumentar a quantidade de séries temporais no dataset. O
tamanho máximo que eu consegui testar no meu notebook sem ele morrer foi 150000 amostras.
'''


# Dataset size
DataSize = 10

# Serie size
SerieSize = 200

# Measurement resolution
resolution = 3

freqs = []
fft_data = []
pdm_data = []
total_interp = 0

# Calls necessary functions to create the dataset
for i in range(DataSize):
    amplitude = np.round(np.random.uniform(0.5, 1.5), 2)
    w0 = np.round(np.random.uniform(1, 9), 2)
    
    x, y = sinusoidal(amplitude, w0, SerieSize, resolution)
    
    interp_start = time.time()
    fft_data.append(interpolation(x, y, resolution))
    interp_end = time.time()
    total_interp+= (interp_end-interp_start)
    
    pdm_data.append((x, y))
    freqs.append(w0)

fft_data = pd.DataFrame(fft_data).T

print(f"Tempo gasto interpolando a função: {total_interp} s")

### Fourier Transform

In [ ]:
from scipy.fftpack import fft, fftfreq

In [ ]:
def fourierTransform(interpData):
    """ calculates the points of the function in the frequency domain

    Args:
        interpolationData (dataframe): interpolated y points

    Returns:
        list: list of tulples containing the x and y points of the function in the frequency domain
    """
    
    fourierSeries = []
    T = 1/10**resolution

    # loop to cycle through all time series in the dataset
    for i in interpData:
        
        # Remove NaN values from the dataframe
        y_interpolation = interpData[i].dropna().values
        
        N = len(y_interpolation)
        
        # Here we apply the Forier transformation
        yf = fft(y_interpolation)[:int(N/2)]
        xf = fftfreq(N, T)[:int(N/2)]
        
        fourierSeries.append((xf,yf))
        
    return fourierSeries

In [ ]:
def calcFreq():
    """ Calculates the peak frequency and the period of the system

    Returns:
        list: frequency and period calculateds 
    """
    
    transf_data = fourierTransform(fft_data)
    fft_results = []
    
    # loop to cycle through all time series in the dataset
    for i in range(len(transf_data)):
        
        xf = transf_data[i][0]
        yf = transf_data[i][1]
        
        # calculates the peak frequency
        best_frequencia = xf[np.absolute(yf).argmax()]
        
        # calculates the period
        best_periodo = np.round(1/best_frequencia, resolution)

        fft_results.append((best_frequencia, best_periodo))
    
    return fft_results



In [ ]:
""" 
Em alguns momentos, por algum motivo, a transformada de fourier não está funcionando. Acredito que seja algum problema relacionado à interpolação dos dados, mas
como isso só ocorreu em algumas raras ocasiões, acabei deixando de lado e tentando focar nos algoritmos de machine learning. Mas fica o aviso, caso veja um warning
sobre uma divisão por zero, é por isso.
"""

fft_start = time.time()

fft_results = calcFreq()

# loop to cycle through all fft results
for i in range(len(fft_results)):
    
    # calculating frequency
    w0 = np.round(2*np.pi*fft_results[i][0], resolution)
    expected_w0 = freqs[i]
    w0_error =  abs(np.round(((w0 - expected_w0)/expected_w0)*100, resolution))
    
    # calculating period
    period = np.round(2*np.pi / w0, resolution)
    expected_p = np.round(2*np.pi / expected_w0,resolution)
    p_error =  abs(np.round(((period - expected_p)/expected_p)*100, resolution))
    
    # printing results
    print(f"w calculated: {w0}, expected w: {expected_w0} and error: {w0_error}% ")
    print(f"Period calculated: {period}, expected period: {expected_p} and error: {p_error}% \n")
    print("\n")

fft_end = time.time()
fft_spent = fft_end-fft_start

### PDM - Phase Dispersion Minimization

In [ ]:
from PyAstronomy.pyTiming import pyPDM

In [ ]:
def calc_pdm(pdm_data):
    """ Use the PDM method to calculate the system period

    Args:
        pdm_data (list): original sinusoidal function without interpolation
    """
    
    n_bins = 10
    pdm_freqs = []
    
    # loop to cycle through all time series in the dataset
    for i in range(len(pdm_data)):
        
        # implements PDM method
        P = pyPDM.PyPDM(pdm_data[i][0], pdm_data[i][1])
        scan = pyPDM.Scanner(minVal=1/10**resolution, maxVal=int(SerieSize/3), 
                            dVal=1/10**resolution, mode="frequency")

        # calculates the frequency and theta
        frequencia, theta = P.pdmEquiBinCover(n_bins, 5, scan)
        
        # append the best frequency
        pdm_freqs.append((2*np.pi*frequencia[theta.argmin()]))
    
    return pdm_freqs

In [ ]:
pdm_start = time.time()

pdm_results = calc_pdm(pdm_data)

# loop to cycle through all pdm results
for i in range(len(pdm_results)):
    
    # calculating frequency
    w0 = np.round(pdm_results[i], resolution)
    expected_w0 = freqs[i]
    w0_error =  abs(np.round(((w0 - expected_w0)/expected_w0)*100, resolution))
    
    # calculating period
    period = np.round(2*np.pi / w0, resolution)
    expected_p = np.round(2*np.pi / expected_w0 ,resolution)
    p_error =  abs(np.round(((period - expected_p)/expected_p)*100, resolution))
    
    # printing results
    print(f"w calculated: {w0}, expected w: {expected_w0} and error: {w0_error}% ")
    print(f"Period calculated: {period}, expected period: {expected_p} and error: {p_error}% \n")
    
pdm_end = time.time()
pdm_spent = pdm_end-pdm_start

#### Avaliando o desempenho

In [ ]:
# Total time spent in method one
total_fft = total_interp + fft_spent
print("Method one: Interpolation + Fourier transform")
print(f"Total time spent: {total_fft} s\n")

# Total time spent in method two
total_pdm= pdm_spent
print("Method two: PDM")
print(f"Total time spent: {total_pdm} s")

Bom, se tudo ocorreu bem, você irá perceber que o tempo total gasto para interpolar a função e depois calcular a transformada de Fourier é muitas vezes mais rápido do que calcular pelo método PDM. Vale destacar também que o erro encontrado no método 1 é bem pequeno.

### Algoritmo de regressão (Extra)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

#### Linear Regression

Bom, aqui segue minha falha tentativa de implementar um algoritmo de regressão. O melhor resultados que eu obtide, executando o LinearRegression, foi:

Erro Médio Quadrado (MSE): 3.813668498412939
Erro Absoluto Médio (MAE): 1.6037934491453092
Coeficiente de Determinação (R-squared): 0.28300592960334636

Para obter esse resultado, utilizei 150000 amostras, diminuindo a resolução para uma casa decimal. Acredito que aumentando mais o número de amostras, o resultado deva melhorar. Infelizmente, meu notebook simplesmente mata o processo.

In [ ]:
# Input time series
input_data = fft_data.fillna(0)

# Corresponding output float values
output_values = freqs

# Convert lists into a numpy array
X = np.array(input_data.T)
y = np.array(output_values)

# Scale the training data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.4, random_state=42)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test data
predictions = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Print the evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Coefficient of Determination (R-squared): {r2}")


#### SVR

Esse modelo, ao contrário do primeiro, aplica uma regressão não linear. O problema é que ele demora mais para ser treinado. Deixei rodando por quase duas horas, com uma série reduzida, e mesmo assim ele não terminou de ser executado. Na teoria, ele deveria apresentar um resultado melhor que o anterior. Caso consiga rodar ele na sua máquina, me avise sobre os resultados, eu fiquei curioso.

In [ ]:
# Input time series
input_data = fft_data.fillna(0)

# Corresponding output float values
output_values = freqs

# Convert the time series into a numpy array
X = np.array(input_data.T)

# The model's target outputs
y = np.array(output_values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create and train the SVR model
model_svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)  # 'rbf' is the radial basis function (Gaussian) kernel
model_svr.fit(X_train, y_train)

# Make predictions on the test data
predictions_svr = model_svr.predict(X_test)

# Calculate evaluation metrics for the SVR model
mse_svr = mean_squared_error(y_test, predictions_svr)
mae_svr = mean_absolute_error(y_test, predictions_svr)
r2_svr = r2_score(y_test, predictions_svr)

# Print the evaluation metrics for the SVR model
print("Evaluation Metrics for the SVR Model:")
print(f"Mean Squared Error (MSE): {mse_svr}")
print(f"Mean Absolute Error (MAE): {mae_svr}")
print(f"Coefficient of Determination (R-squared): {r2_svr}")


### Conclusão

Embora não tenha sido possível obter resultados satisfatórios utilizando os algoritmos de regressão, o uso de interpolação somado à transformada de fourier parece ser promissor e mostrou reduzir significativamente o tempo de execução quando comparado ao PDM.

Como possíveis incrementos desse projeto, destaca-se o treinamento dos algoritmos usados acima com um dataset mais robusto para identificar os seus reais potenciais. O uso de outros algoritmos de aprendizado, como algoritmos de redes neurais, também parece ser promissor para o caso. 

